In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

class Normalize(nn.Module):

    def __init__(self, power=2):
        super(Normalize, self).__init__()
        self.power = power
    
    def forward(self, x):
        norm = x.pow(self.power).sum(1, keepdim=True).pow(1./self.power)
        out = x.div(norm)
        return out

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, low_dim=128):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, low_dim)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        #x = self.fc(x)
        x = F.normalize(x)
        return x


def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model


def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model


In [2]:
availableBackends = {'resnet18':resnet18, 'resnet34':resnet34, 'resnet50':resnet50, 'resnet101':resnet101,'resnet152':resnet152}

class MLP(nn.Module):
    def __init__(self,
    input_size = 2048,
    hidden_size = 4096,
    output_size = 256,
    depth = 2,
    ):  
        super().__init__()
        layers = []
        inp = input_size
        for d in range(depth):
            if d == depth - 1:
                layers.append(nn.Linear(inp, output_size))
            else:
                layers.extend([nn.Linear(inp, hidden_size), nn.BatchNorm1d(hidden_size), nn.ReLU(inplace=True)])
                inp = hidden_size
        self.layer = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layer(x)

class OnlineNetwork(nn.Module):
    def __init__(self,
    input_size=2048,
    hidden_size=4096,
    output_size=256,
    depth_proj=2,
    depth_pred=2,
    closedFormPredicator = False,
    backend = 'resnet50',
    pretrained = False):
        super().__init__()
        self.backend = availableBackends[backend](pretrained=pretrained)
        self.projection = MLP(input_size = input_size, output_size = output_size, hidden_size = hidden_size, depth = depth_proj)
        self.predictor = None

        if not closedFormPredicator:
            self.predictor = MLP(input_size=output_size, output_size = output_size, hidden_size = hidden_size, depth = depth_pred)

    def forward(self, x):
        x = self.backend(x)
        out = self.projection(x)

        if self.predictor is not None:
            out = self.predictor(out)
        
        return out


class TargetNetwork(nn.Module):
    def __init__(self,
    input_size=2048,
    hidden_size=4096,
    output_size=256,
    depth_proj=2,
    backend = 'resnet50',
    pretrained = False):
        super().__init__()
        self.backend = availableBackends[backend](pretrained=pretrained)
        self.projection = MLP(input_size = input_size, output_size = output_size, hidden_size = hidden_size, depth = depth_proj)
        self.predictor = None


        for model in [self.backend, self.projection]:
            for p in model.parameters():
                p.requires_grad = False

    def forward(self, x):
        out = self.backend(x)
        out = self.projection(out)
        return out

class BYOL(nn.Module):
    def __init__(self,
    input_size=2048,
    hidden_size=4096,
    output_size=256,
    depth_proj=2,
    depth_pred=2,
    closedFormPredicator = False,
    EMA = True,
    t = 0.996,
    backend='resnet50',
    pretrained = False):
        super().__init__()
        self.t = t
        self.EMA = EMA
        self.closedForm = closedFormPredicator
        self.onlineNet = OnlineNetwork(input_size=input_size, hidden_size=hidden_size, output_size=output_size, 
        depth_proj= depth_proj, depth_pred=depth_pred, closedFormPredicator = closedFormPredicator, backend=backend, pretrained=pretrained)
        self.targetNet = TargetNetwork(input_size=input_size, hidden_size=hidden_size, output_size=output_size, depth_proj=depth_proj, backend=backend, pretrained=pretrained)

    def updateTargetNetwork(self, lamb = 10):
        with torch.no_grad():
            for pOn,pTa in zip(self.onlineNet.parameters(), self.targetNet.parameters()):
                if self.EMA and not self.closedForm :
                    pTa = self.t*pTa + (1-self.t)*pOn
                else:
                    pTa = lamb*pOn

    def forward(self, x, y):
        xOn = self.onlineNet(x)
        yOn = self.onlineNet(y)
        xTg = self.targetNet(x)
        yTg = self.targetNet(y)

        if self.closedForm:
            xOn = (xOn.T@yTg)/(xOn.T@xOn)
            yOn = (yOn.T@xTg)/(yOn.T@yOn)

        return xOn, yOn, xTg, yTg

In [14]:
import torchvision.transforms as tr

def regression_loss(x, y):
    x = F.normalize(x, dim=1, p=2)
    y = F.normalize(y, dim=1, p=2)
    return 2 - 2 * (x*y).sum(dim=-1)

def criterion(xOn, yTg, yOn, xTg):
    return (regression_loss(xOn, yTg) + regression_loss(yOn, xTg)).mean()


def get_byol_transforms(size, mean, std):
    transformT = tr.Compose([
    tr.RandomResizedCrop(size=size, scale=(0.08,1), ratio=(3 / 4, 4 / 3)),
    tr.RandomApply(nn.ModuleList([tr.RandomRotation((-90, 90))]), p=0.5),
    tr.RandomApply(nn.ModuleList([tr.ColorJitter()]), p=0.8),
    tr.GaussianBlur(kernel_size=(23,23), sigma=(0.1, 2.0)),
    #tr.RandomGrayscale(p=0.2),
    tr.Normalize(mean, std)])

    transformT1 = tr.Compose([
        tr.RandomResizedCrop(size=size, scale=(0.08,1), ratio=(3 / 4, 4 / 3)),
        tr.RandomApply(nn.ModuleList([tr.RandomRotation((-90, 90))]), p=0.5),
        tr.RandomApply(nn.ModuleList([tr.ColorJitter()]), p=0.8),
        #tr.RandomGrayscale(p=0.2),
        tr.RandomApply(nn.ModuleList([tr.GaussianBlur(kernel_size=(23,23), sigma=(0.1, 2.0))]), p=0.1),
        tr.Normalize(mean, std)])

    transformEvalT = tr.Compose([
        tr.CenterCrop(size=size),
        tr.Normalize(mean, std)
    ])

    return transformT, transformT1, transformEvalT

In [15]:
import torch.optim as optim
import torchvision.transforms as tr
import torchvision.datasets as datasets
import numpy as np
from tqdm import tqdm

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

epochs = 30
batch_size = 512
offset_bs = 256
base_lr = 0.03
tempBase = 0.996

transformT, transformT1, transformEvalT = get_byol_transforms(32, (0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

#traindt = datasets.ImageNet(root='./data', split = 'train')
#trainloader = torch.utils.data.DataLoader(traindt, batch_size=batch_size, shuffle=True)

#testdt = datasets.ImageNet(root='./data', split = 'val')
#testloader = torch.utils.data.DataLoader(traindt, batch_size=128, shuffle=True)

trainset = datasets.STL10(root='./', split='unlabeled', transform=tr.ToTensor(), download=True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

lr = base_lr*batch_size/offset_bs

byol = BYOL(input_size=512, closedFormPredicator=False, backend='resnet34')

byol.to(device)

params = byol.parameters()
optimizer = optim.SGD( params, lr=lr, weight_decay=1.5e-4)

def train_loop(model, optimizer, trainloader, transform, transform1, criterion, device):
    tk0 = tqdm(trainloader)
    train_loss = []
    var_loss = []

    for batch, inf in tk0:
        #print(inf)
        batch = batch.to(device)
        
        x = transform(batch)
        x1 = transform1(batch)

        onlinex, onlinex1, targetx, targetx1 = model(x, x1)
        loss = criterion(onlinex, targetx1, onlinex1, targetx)
        train_loss.append(loss.item())
        with torch.no_grad():
          var_online = onlinex.var(dim=0).mean()
          var_target = targetx.var(dim=0).mean()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        byol.updateTargetNetwork()
        tk0.set_description(f"the variance in online: {var_online} Variance target: {var_target}")

        del batch, x, x1, onlinex, onlinex1, targetx, targetx1
    return train_loss


for epoch in range(epochs):
    train_loss = train_loop(byol, optimizer, trainloader, transformT, transformT1, criterion, torch.device('cuda'))
    print(f"Epoch: {epoch} train_loss: {np.mean(train_loss)}")


Files already downloaded and verified


the variance in online: 0.25450727343559265 Variance target: 0.10934548825025558: 100%|██████████| 196/196 [02:14<00:00,  1.46it/s]


Epoch: 0 train_loss: 1.725413658789226


the variance in online: 0.2922554612159729 Variance target: 0.10910944640636444: 100%|██████████| 196/196 [02:14<00:00,  1.46it/s] 


Epoch: 1 train_loss: 1.685900741693925


...

Epoch: 27 train_loss: 1.6655894311106936


the variance in online: 0.481828510761261 Variance target: 0.1090821921825409: 100%|██████████| 196/196 [02:13<00:00,  1.46it/s]  


Epoch: 28 train_loss: 1.6638259620082623


the variance in online: 0.5968812704086304 Variance target: 0.10986419767141342: 100%|██████████| 196/196 [02:13<00:00,  1.46it/s]

Epoch: 29 train_loss: 1.6667980819332355


In [21]:
class LogisticRegression(nn.Module):
    def __init__(self, encoder, input_size, output_size):
        super().__init__()
        
        self.encoder = encoder
        self.linear = nn.Linear(input_size, output_size)
        
        for p in self.encoder.parameters():
            p.required_grad = False
        
    
    def forward(self, x):
        self.encoder.eval()
        x = self.encoder(x)
        x = self.linear(x)
        
        return x

In [26]:
trainset = datasets.STL10(root='./', split='train', transform=tr.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True)

testset = datasets.STL10(root='./', split='test', transform=tr.ToTensor(), download=True)
test_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

model = LogisticRegression(byol.onlineNet.backend, 512, 10).to(device)

Files already downloaded and verified
Files already downloaded and verified


In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10

for epoch in range(200):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = model(x)
        predictions = torch.argmax(logits, dim=1)
        
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 85.26
Testing accuracy: 100.0


KeyboardInterrupt: 